In [1]:
import pandas as pd
import gripql
from tqdm import tqdm
import netvae

from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

ModuleNotFoundError: No module named 'netvae'

In [ ]:
tf_min_size = 3
learning_rate=1e-4
batch_size=256
epochs=50
latent_dim = 768
rotation_factor = 0.3

In [3]:
# get the sheet from https://cancer.sanger.ac.uk/cosmic/census?tier=all
genesDF = pd.read_csv("muta.tsv", sep="\t", index_col=0)

In [4]:
#geneList = genesDF[ (genesDF["Somatic"] == "yes") & (genesDF["Hallmark"] == "Yes") ].index.to_list()
geneList = genesDF[ (genesDF["Somatic"] == "yes") ].index.to_list()

In [5]:
conn = gripql.Connection(url='https://bmeg.io/grip', credential_file='../bmeg_credentials.json')
G = conn.graph("rc6")

In [6]:
geneMap = dict( (i[0],i[1]) for i in G.query().V().hasLabel("Gene").render(["gene_id", "symbol"]).execute() )

In [7]:
hugoMap = {}
for k,v in geneMap.items():
    if v in geneList:
        hugoMap[v] = k

In [8]:
projects = G.query().V('Program:TCGA').out("projects").render("$._gid").execute()

In [9]:
targetList = list(hugoMap.values())

In [10]:
data = {}
for p in tqdm(projects):
    q = G.query().V(p).out("cases").out("samples").as_("sample")
    q = q.has(gripql.eq("$.gdc_attributes.sample_type", "Primary Tumor"))
    q = q.out("aliquots").out("somatic_callsets")
    q = q.outE().has(gripql.within("ensembl_gene", targetList))
    q = q.as_("variant")
    q = q.render(["$sample.submitter_id", "$variant.ensembl_gene"])
    for sample, gene in q:
        if sample not in data:
            data[sample] = {gene:1}
        else:
            data[sample][gene] = 1

100%|███████████████████████████████████████████████████████████████████████████████████████| 33/33 [01:19<00:00,  2.42s/it]


In [11]:
X = pd.DataFrame(data).rename(index=geneMap).fillna(0).transpose()

In [12]:
X

,KRAS,TET1,CYSLTR2,PML,BRCA2,TP53,PIK3CA,HIST1H4I,CCNC,FBXW7,...,SUB1,ISX,SUFU,EED,NDRG1,FSTL3,FCGR2B,SLC45A3,RMI2,RAP1B
TCGA-AG-A00C-01A,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-EI-6917-01A,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-DY-A0XA-01A,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-F5-6810-01A,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-AG-A01W-01A,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-VQ-A8PD-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-MX-A5UJ-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
TCGA-BR-6564-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-BR-4191-01A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
epochs=250


X_encoder = netvae.build_encoder(X.shape[1], latent_dim)
X_decoder = netvae.build_decoder(X.shape[1], latent_dim)
X_vae = netvae.VAE(X_encoder, X_decoder)
X_vae.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate))
history = X_vae.fit(X, epochs=epochs, batch_size=batch_size, shuffle=True)

2024-08-01 13:27:36.455992: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-08-01 13:27:36.456023: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-08-01 13:27:36.456032: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-08-01 13:27:36.456069: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-01 13:27:36.456084: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 657)]                0         []                            
                                                                                                  
 pathway_layer (Dense)       (None, 768)                  505344    ['input_1[0][0]']             
                                                                                                  
 batchnorm (BatchNormalizat  (None, 768)                  3072      ['pathway_layer[0][0]']       
 ion)                                                                                             
                                                                                                  
 z_mean (Dense)              (None, 768)                  590592    ['batchnorm[0][0]']     

None
Epoch 1/250


2024-08-01 13:27:37.159445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


37/37 [==============================] - 2s 31ms/step - loss: 530.3608 - reconstruction_loss: 496.3244 - kl_loss: 13.5592
Epoch 2/250
37/37 [==============================] - 0s 11ms/step - loss: 412.7623 - reconstruction_loss: 339.6320 - kl_loss: 28.1311
Epoch 3/250
37/37 [==============================] - 0s 12ms/step - loss: 239.8156 - reconstruction_loss: 163.9230 - kl_loss: 53.9169
Epoch 4/250
37/37 [==============================] - 0s 13ms/step - loss: 169.0732 - reconstruction_loss: 110.5033 - kl_loss: 50.2876
Epoch 5/250
37/37 [==============================] - 0s 11ms/step - loss: 138.4125 - reconstruction_loss: 90.5012 - kl_loss: 44.2071
Epoch 6/250
37/37 [==============================] - 0s 12ms/step - loss: 121.7434 - reconstruction_loss: 80.2751 - kl_loss: 39.5041
Epoch 7/250
37/37 [==============================] - 0s 12ms/step - loss: 113.1303 - reconstruction_loss: 74.7335 - kl_loss: 35.8925
Epoch 8/250
37/37 [==============================] - 0s 11ms/step - loss: 105

37/37 [==============================] - 0s 9ms/step - loss: 49.7648 - reconstruction_loss: 41.2386 - kl_loss: 7.8154
Epoch 126/250
37/37 [==============================] - 0s 9ms/step - loss: 48.9892 - reconstruction_loss: 41.1166 - kl_loss: 7.8158
Epoch 127/250
37/37 [==============================] - 0s 9ms/step - loss: 49.2155 - reconstruction_loss: 40.8949 - kl_loss: 7.8488
Epoch 128/250
37/37 [==============================] - 0s 9ms/step - loss: 48.4930 - reconstruction_loss: 40.8605 - kl_loss: 7.8382
Epoch 129/250
37/37 [==============================] - 0s 9ms/step - loss: 48.0069 - reconstruction_loss: 40.7456 - kl_loss: 7.8715
Epoch 130/250
37/37 [==============================] - 0s 9ms/step - loss: 47.7505 - reconstruction_loss: 40.6151 - kl_loss: 7.8807
Epoch 131/250
37/37 [==============================] - 0s 9ms/step - loss: 49.1838 - reconstruction_loss: 40.4392 - kl_loss: 7.8879
Epoch 132/250
37/37 [==============================] - 0s 9ms/step - loss: 49.4178 - recon

37/37 [==============================] - 0s 9ms/step - loss: 41.8149 - reconstruction_loss: 32.4385 - kl_loss: 9.2384
Epoch 188/250
37/37 [==============================] - 0s 10ms/step - loss: 41.8481 - reconstruction_loss: 32.3276 - kl_loss: 9.2668
Epoch 189/250
37/37 [==============================] - 0s 8ms/step - loss: 41.5610 - reconstruction_loss: 32.1645 - kl_loss: 9.2868
Epoch 190/250
37/37 [==============================] - 0s 8ms/step - loss: 41.4276 - reconstruction_loss: 32.0202 - kl_loss: 9.3152
Epoch 191/250
37/37 [==============================] - 0s 9ms/step - loss: 41.9995 - reconstruction_loss: 31.9406 - kl_loss: 9.3450
Epoch 192/250
37/37 [==============================] - 0s 9ms/step - loss: 41.7185 - reconstruction_loss: 31.8352 - kl_loss: 9.3531
Epoch 193/250
37/37 [==============================] - 0s 12ms/step - loss: 41.0655 - reconstruction_loss: 31.7095 - kl_loss: 9.3927
Epoch 194/250
37/37 [==============================] - 0s 9ms/step - loss: 41.4423 - rec

In [14]:
X_pred = pd.DataFrame( X_vae.decoder.predict( X_vae.encoder.predict(X)[0] ), index=X.index, columns=X.columns ).round()

 25/294 [=>............................] - ETA: 0s 

2024-08-01 13:29:03.362635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 93/294 [========>.....................] - ETA: 0s

2024-08-01 13:29:05.696917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


294/294 [==============================] - 1s 2ms/step


In [15]:
i=8
sum(X.iloc[i] == X_pred.iloc[i]) / X.shape[1]

0.9939117199391172

In [16]:
X.shape

(9391, 657)

In [18]:
X_vae.encoder.predict(X)[0]

294/294 [==============================] - 1s 2ms/step


array([[-0.01791587, -0.0791906 ,  0.18129964, ..., -0.08360282,
         0.14226487,  0.19072601],
       [ 0.06319378, -0.16567634, -0.78152347, ...,  0.16215864,
        -1.0181475 , -0.01571302],
       [ 0.1189163 , -0.00389188,  0.04327488, ..., -0.19779831,
        -0.10879896,  0.0977457 ],
       ...,
       [-0.13054757,  0.11437189,  0.08394529, ..., -0.2223345 ,
        -0.14266002, -0.03218059],
       [ 0.1231217 ,  0.10812236,  0.16982532, ..., -0.06853131,
        -0.15471339,  0.02864742],
       [ 0.06914293, -0.05472783,  0.25650588, ..., -0.01729879,
        -0.05574045,  0.03449553]], dtype=float32)